In [ ]:
# GC mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ディレクトリ作成（tfrecord,train_data,val_data）
%cd /content
!mkdir train_data
!mkdir val_data

In [ ]:
# tfrecordの内容を展開(あらかじめGdrive内にデータセットをDLしておく)
!cp "/content/drive/MyDrive/Colab Notebooks/note_Axross/_data/tfrecord.zip" .
!unzip tfrecord.zip -d tfrecord

In [ ]:
# 学習データの内容を展開
!cp "/content/drive/MyDrive/Colab Notebooks/note_Axross/_data/test_data.zip" .
!unzip test_data.zip

In [ ]:
# Protocol Buffersのインストール
!curl -OL https://github.com/google/protobuf/releases/download/v3.15.6/protoc-3.15.6-linux-x86_64.zip
!unzip protoc-3.15.6-linux-x86_64.zip -d protoc3
!sudo mv protoc3/bin/* /usr/local/bin/
!sudo mv protoc3/include/* /usr/local/include/
!rm -rf protoc3 protoc-3.15.6-linux-x86_64.zip

In [ ]:
# ライブラリのインストール
!pip install "opencv-python-headless<4.3"

In [ ]:
# tensorflow/modelsのダウンロード
!git clone --depth 1 https://github.com/tensorflow/models
%cd /content/models/research
!/usr/local/bin/protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# Object Detection APIに必要なライブラリのインストール
!cp /content/models/research/object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
# インストールの成否確認（Object Detection API）
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
# 学習データと評価データの分割

# データ格納先の定義
original_data_dir = '/content/tfrecord'
train_data_dir = '/content/train_data'
val_data_dir = '/content/val_data'

# ファイル数カウント
import glob
file_count = len(glob.glob(original_data_dir + '/*.tfrecord'))
print('File count : ' + str(file_count))

# 学習データ/検証データを80%：20%の割合で分割
import random
import shutil

train_ratio = 0.80

file_list = glob.glob(original_data_dir + '/*.tfrecord')
random_sample_list = random.sample(file_list, file_count)

# ディレクトリへコピー
for index, filepath in enumerate(random_sample_list):
    if index < int(file_count * train_ratio):
        # 学習データ
        shutil.copy2(filepath, train_data_dir)
    else:
        # 検証データ
        shutil.copy2(filepath, val_data_dir)

In [ ]:
# 事前学習済モデルのデータ格納先
%cd /content
!mkdir pretrained_model

In [ ]:
# 事前学習済モデルのダウンロード
!wget -P pretrained_model http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
!tar -xvf pretrained_model/efficientdet_d0_coco17_tpu-32.tar.gz -C pretrained_model
!rm pretrained_model/efficientdet_d0_coco17_tpu-32.tar.gz

In [ ]:
# パイプラインコンフィグの置換
!cp "/content/drive/MyDrive/Colab Notebooks/note_Axross/_data/pipeline.config" "/content/pretrained_model/efficientdet_d0_coco17_tpu-32"

In [ ]:
# モデルのチェックポイント格納ディレクトリ作成
%cd /content
!mkdir model_check_point

In [ ]:
# Tensorboard起動
%load_ext tensorboard

In [ ]:
# Tensorboard起動
tensorboard --logdir /content/model_check_point

In [ ]:
# モデル訓練
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path="/content/pretrained_model/efficientdet_d0_coco17_tpu-32/pipeline.config" \
    --model_dir="/content/model_check_point" \
    --num_train_steps=1000 \

In [ ]:
# モデルのエクスポート
!python /content/models/research/object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path="/content/pretrained_model/efficientdet_d0_coco17_tpu-32/pipeline.config" \
    --trained_checkpoint_dir="/content/model_check_point" \
    --output_directory="/content/model_check_point/output"

In [ ]:
# モデルの読込
import tensorflow as tf
model_path = '/content/model_check_point/output/saved_model'
DEFAULT_FUNCTION_KEY = 'serving_default'
loaded_model = tf.saved_model.load(model_path)
inference_func = loaded_model.signatures[DEFAULT_FUNCTION_KEY]

In [ ]:
# テストデータの閲覧（推論対象のテスト画像）
import glob
import copy
import cv2
from google.colab.patches import cv2_imshow

test_data_dir = '/content/test_data'
testfile_list = sorted(glob.glob(test_data_dir + '/*.jpg'))

In [ ]:
# 推論（推論用関数）
def run_inference_single_image(image, inference_func):
    tensor = tf.convert_to_tensor(image)
    output = inference_func(tensor)

    output['num_detections'] = int(output['num_detections'][0])
    output['detection_classes'] = output['detection_classes'][0].numpy()
    output['detection_boxes'] = output['detection_boxes'][0].numpy()
    output['detection_scores'] = output['detection_scores'][0].numpy()
    return output

In [ ]:
# 結果動画の書出し
import cv2
import numpy as np
from PIL import Image

filenames = []

temp_image = cv2.imread(testfile_list[0], cv2.IMREAD_UNCHANGED)
image_width, image_height = temp_image.shape[1], temp_image.shape[0]
fourcc = 'mp4v'
writer_fourcc = cv2.VideoWriter_fourcc(*fourcc)
videowriter = cv2.VideoWriter('result.mp4', writer_fourcc, 10, (image_width, image_height))

# 推論
for filecount, testfile in enumerate(testfile_list):
    image = cv2.imread(testfile, cv2.IMREAD_UNCHANGED)
    debug_image = copy.deepcopy(image)

    image_width, image_height = image.shape[1], image.shape[0]
    image = image[:, :, [2, 1, 0]]  # BGR形式からRGB形式へ変換
    image_np_expanded = np.expand_dims(image, axis=0)

    output = run_inference_single_image(image_np_expanded, inference_func)

    num_detections = output['num_detections']
    for i in range(num_detections):
        score = output['detection_scores'][i]
        bbox = output['detection_boxes'][i]
        # class_id = output['detection_classes'][i].astype(np.int)

        if score < 0.85:
            continue

        x1, y1 = int(bbox[1] * image_width), int(bbox[0] * image_height)
        x2, y2 = int(bbox[3] * image_width), int(bbox[2] * image_height)

        # 推論結果描画
        cv2.rectangle(debug_image, (x1, y1), (x2, y2), (255, 255, 255), 2)
        cv2.putText(debug_image, str('{:.2f}'.format(score)), (x1, y1-10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.rectangle(debug_image, (x1, y1), (x2, y2), (255, 255, 255), 2)
    videowriter.write(debug_image)
videowriter.release()

In [ ]:
# 結果動画の再生
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def play_video(video, interval=100):
    video = imageio.mimread(video)
    fig = plt.figure(figsize=(9, 6))

    movie = []
    for i in range(len(video)):
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        movie.append([img])

    anime = animation.ArtistAnimation(fig, movie, interval=interval, repeat_delay=1000)
    plt.close()
    return anime

HTML(play_video('result.mp4').to_html5_video())